In [0]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [0]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

trainfile = r'/gdrive/My Drive/DataMining-1/Assignment - 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/DataMining-1/Assignment - 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 

print(test_data.shape)
print(test_data.head()) 

(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]
(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [0]:
#Copy Train data excluding target
trainData_Copy = train_data.iloc[:,:-1].copy()
#Copy Test data excluding GeographicalField64 column
testData_Copy = test_data.iloc[:,:-1].copy()

#Separate Train data and test data
X_train = trainData_Copy
X_test = testData_Copy
#Selecting only target columm from Training dataset
y_train=train_data.iloc[:,-1] 




print(X_train.head())
print(X_test.head()) 
print(y_train.head())
 

   CoverageField11A  ...  GeographicField64_TX
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     1
4                 4  ...                     0

[5 rows x 595 columns]
   CoverageField11A  ...  GeographicField64_TX
0                13  ...                     0
1                 4  ...                     0
2                 3  ...                     0
3                 5  ...                     0
4                12  ...                     0

[5 rows x 595 columns]
0    0
1    0
2    0
3    0
4    0
Name: QuoteConversion_Flag, dtype: int64


In [0]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict_train=clf.predict(X_train)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("Confusion Matrix for training Decision Tree")
print(confusion_matrix(y_train,clf_predict_train))
print("Classification report for training Decision Tree")
print(classification_report(y_train,clf_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction and corresponding probability
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/DefaultDecisionTree_Test_results.csv", index = None)

accuracy Score (training) for Decision Tree:1.000000
Confusion Matrix for training Decision Tree
[[52738     0]
 [    0 12262]]
Classification report for training Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     12262

    accuracy                           1.00     65000
   macro avg       1.00      1.00      1.00     65000
weighted avg       1.00      1.00      1.00     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     1


In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

{'min_samples_split': 70, 'max_depth': 7}


In [0]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict_train=clf.predict(X_train)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))
print("Confusion Matrix for training Decision Tree")
print(confusion_matrix(y_train,clf_predict_train))
print("Classification report for training Decision Tree")
print(classification_report(y_train,clf_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(clf.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())



accuracy Score (training) for Decision Tree:0.912231
Confusion Matrix for training Decision Tree
[[52204   534]
 [ 5171  7091]]
Classification report for training Decision Tree
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     52738
           1       0.93      0.58      0.71     12262

    accuracy                           0.91     65000
   macro avg       0.92      0.78      0.83     65000
weighted avg       0.91      0.91      0.90     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict_train=rfc.predict(X_train)
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_train,y_train)))
print("Confusion Matrix for training Random Forest")
print(confusion_matrix(y_train,rfc_predict_train))
print("Classification report for training Random Forest")
print(classification_report(y_train,rfc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())





In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

{'n_estimators': 130, 'min_samples_split': 20, 'max_depth': 19}


In [0]:


#contruct random forest using the best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict_train=rfc.predict(X_train)
print("accuracy Score (training) for Random Forest:{0:6f}".format(rfc.score(X_train,y_train)))
print("Confusion Matrix for training Random Forest")
print(confusion_matrix(y_train,rfc_predict_train))
print("Classification report for training Random Forest")
print(classification_report(y_train,rfc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred=pd.DataFrame(rfc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/RandomForestHyper_beforeTuning_Test_results.csv", index = None)




accuracy Score (training) for Random Forest:0.954277
Confusion Matrix for training Random Forest
[[52633   105]
 [ 2867  9395]]
Classification report for training Random Forest
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     52738
           1       0.99      0.77      0.86     12262

    accuracy                           0.95     65000
   macro avg       0.97      0.88      0.92     65000
weighted avg       0.96      0.95      0.95     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#K_Nearest Neighbour============================================================

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
knn_predict_train=knn.predict(X_train)
print("accuracy Score (training) for KNN:{0:6f}".format(knn.score(X_train,y_train)))
print("Confusion Matrix for training KNN")
print(confusion_matrix(y_train,knn_predict_train))
print("Classification report for training KNN")
print(classification_report(y_train,knn_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(knn.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/KNNHyper_beforeTuning_Test_results.csv", index = None)

accuracy Score (training) for KNN:0.829215
Confusion Matrix for training KNN
[[51551  1187]
 [ 9914  2348]]
Classification report for training KNN
              precision    recall  f1-score   support

           0       0.84      0.98      0.90     52738
           1       0.66      0.19      0.30     12262

    accuracy                           0.83     65000
   macro avg       0.75      0.58      0.60     65000
weighted avg       0.81      0.83      0.79     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#SVM Classifier============================================================

svmc = LinearSVC() # Linear Kernel
svmc.fit(X_train,y_train)
svmc_predict_train=svmc.predict(X_train)
print("accuracy Score (training) for SVM:{0:6f}".format(svmc.score(X_train,y_train)))
print("Confusion Matrix for training SVM")
print(confusion_matrix(y_train,svmc_predict_train))
print("Classification report for training SVM")
print(classification_report(y_train,svmc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(svmc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/SVMBeforeTuning_Test_results.csv", index = None)

accuracy Score (training) for SVM:0.797508
Confusion Matrix for training SVM
[[50749  1989]
 [11173  1089]]
Classification report for training SVM
              precision    recall  f1-score   support

           0       0.82      0.96      0.89     52738
           1       0.35      0.09      0.14     12262

    accuracy                           0.80     65000
   macro avg       0.59      0.53      0.51     65000
weighted avg       0.73      0.80      0.75     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#Gradient Boosting ============================================================

abc =GradientBoostingClassifier()
abc.fit(X_train,y_train)
abc_predict_train=abc.predict(X_train)
print("accuracy Score (training) for Gradient Boosting:{0:6f}".format(abc.score(X_train,y_train)))
print("Confusion Matrix for training Gradient Boosting")
print(confusion_matrix(y_train,abc_predict_train))
print("Classification report for training Gradient Boosting")
print(classification_report(y_train,abc_predict_train))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(abc.predict(X_test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())


accuracy Score (training) for Gradient Boosting:0.923554
Confusion Matrix for training Gradient Boosting
[[52114   624]
 [ 4345  7917]]
Classification report for training Gradient Boosting
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     52738
           1       0.93      0.65      0.76     12262

    accuracy                           0.92     65000
   macro avg       0.92      0.82      0.86     65000
weighted avg       0.92      0.92      0.92     65000

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using RandomForest,KNN and SVM Classifier\n")

models = [ RandomForestClassifier(),KNeighborsClassifier(),LinearSVC() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestClassifier]
    fold  0:  [0.90904035]
    fold  1:  [0.91515397]
    fold  2:  [0.91277306]
    fold  3:  [0.91287419]
    ----
    MEAN:     [0.91246039] + [0.00219209]
    FULL:     [0.91246034]

model  1:     [KNeighborsClassifier]
    fold  0:  [0.70067752]
    fold  1:  [0.70354452]
    fold  2:  [0.70160801]
    fold  3:  [0.70216424]
    ----
    MEAN:     [0.70199857] + [0.00103864]
    FULL:     [0.70199856]

model  2:     [LinearSVC]
    fold  0:  [0.56552735]
    fold  1:  [0.61783890]
    fold  2:  [0.68072411]
    fold  3:  [0.68886529]
    ----
    MEAN:     [0.63823891] + [0.05017757]
    FULL:     [0.63823682]



In [0]:
#STACKING - CONTRUCT A KNN MODEL==============================
model =  KNeighborsClassifier(n_neighbors=5)
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for KNN after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training KNN after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training KNN after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction 
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/KNNAfterStacking_Test_results.csv", index = None)


accuracy Score (training) for KNN after STACKING:0.828498
Confusion Matrix for training KNN after STACKING
[[51530  1208]
 [12359 14010]]
Classification report for training KNN after STACKING
              precision    recall  f1-score   support

           0       0.81      0.98      0.88     52738
           1       0.92      0.53      0.67     26369

    accuracy                           0.83     79107
   macro avg       0.86      0.75      0.78     79107
weighted avg       0.84      0.83      0.81     79107

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#STACKING - RANDOM FOREST MODEL AFTER TUNING==============================
model =  RandomForestClassifier(**grid_parm_rfc)
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for RANDOM FOREST after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training RANDOM FOREST after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training RANDOM FOREST after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/RandomForestAfterStacking_Test_results.csv", index = None)

accuracy Score (training) for RANDOM FOREST after STACKING:0.912460
Confusion Matrix for training RANDOM FOREST after STACKING
[[51265  1473]
 [ 5452 20917]]
Classification report for training RANDOM FOREST after STACKING
              precision    recall  f1-score   support

           0       0.90      0.97      0.94     52738
           1       0.93      0.79      0.86     26369

    accuracy                           0.91     79107
   macro avg       0.92      0.88      0.90     79107
weighted avg       0.91      0.91      0.91     79107

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0


In [0]:
#STACKING - SVC MODEL AFTER TUNING==============================
model =  LinearSVC()
    
model = model.fit(S_Train, y_res)
y_pred=model.predict(S_Train)
print("accuracy Score (training) for SVM after STACKING:{0:6f}".format(model.score(S_Train,y_res)))
print("Confusion Matrix for training SVM after STACKING")
print(confusion_matrix(y_res,y_pred))
print("Classification report for training RANDOM FORESTSVM after STACKING")
print(classification_report(y_res,y_pred))

#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as QuoteConversion_Flag
pred=pd.DataFrame(model.predict(S_Test),columns=["QuoteConversion_Flag"])
pred.head()

output_col=pd.concat([test_data["QuoteNumber"],pred],axis=1)
print(output_col.head())

#Write into a file with actual prediction
output_col.to_csv("/gdrive/My Drive/DataMining-1/Assignment - 3/SVMAfterStacking_Test_results.csv", index = None)

accuracy Score (training) for SVM after STACKING:0.912460
Confusion Matrix for training SVM after STACKING
[[51265  1473]
 [ 5452 20917]]
Classification report for training RANDOM FORESTSVM after STACKING
              precision    recall  f1-score   support

           0       0.90      0.97      0.94     52738
           1       0.93      0.79      0.86     26369

    accuracy                           0.91     79107
   macro avg       0.92      0.88      0.90     79107
weighted avg       0.91      0.91      0.91     79107

   QuoteNumber  QuoteConversion_Flag
0            3                     0
1            5                     0
2            7                     0
3            9                     0
4           10                     0
